In [12]:
%pip install facenet-pytorch pandas tqdm scikit-learn

^C
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/pip/__main__.py", line 22, in <module>
    from pip._internal.cli.main import main as _main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cl

In [1]:
from facenet_pytorch import MTCNN
import torch
from torch.utils.data import DataLoader
from torchvision import datasets

import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import random

import PIL

RNG_SEED = 42

random.seed(RNG_SEED)
torch.manual_seed(RNG_SEED)
np.random.seed(RNG_SEED)

DATA_DIR = "data"

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


# Define Dataset

In [4]:
try:
    # create set folder
    os.makedirs(f"{DATA_DIR}/test")
    os.makedirs(f"{DATA_DIR}/train")

    # create label folders
    os.makedirs(f"{DATA_DIR}/test/face")
    os.makedirs(f"{DATA_DIR}/test/no_face")
    
    os.makedirs(f"{DATA_DIR}/train/face")
    os.makedirs(f"{DATA_DIR}/train/no_face")
except:
    print("Folders already exist.")

Folders already exist.


In [5]:
from IPython.display import Image

labels = pd.read_csv("labels.csv")

unmoved = [] # this was used to store all images that were to be labeled

files = []
test_files = []
labeled_files = set(labels["filename"].values)

for (dirpath, dirnames, filenames) in os.walk(DATA_DIR):
    files.extend(filenames)
    break

for (dirpath, dirnames, filenames) in os.walk(f"{DATA_DIR}/test"):
    for dirname in dirnames:
        for (dirpath, dirnames, filenames) in os.walk(f"{DATA_DIR}/test/{dirname}"):
            test_files.extend(filenames)
            break
    break

unmoved = labels[-labels["filename"].isin(test_files)]
files = [file for file in files if file not in labeled_files]

In [6]:
len(unmoved)

72219

In [7]:
import ipywidgets as widgets
from IPython.display import Image, display, clear_output

face_bttn = widgets.Button(description="Face")
no_face_bttn = widgets.Button(description="No Face")
out = widgets.Output()

count = [0]

curr_file = ''

def face_bttn_clicked(_):
    d = {'filename': files[0],
                   'label': 'face'}
    files.pop(0)
    labels.loc[len(labels)] = d

    show_widgets()
        
face_bttn.on_click(face_bttn_clicked)

def no_face_clicked(_):
    d = {'filename': files[0],
                   'label': 'no face'}
    files.pop(0)
    labels.loc[len(labels)] = d

    show_widgets()

no_face_bttn.on_click(no_face_clicked)

def show_widgets():
    clear_output(wait=True)
    buttons = widgets.HBox([face_bttn, no_face_bttn])
    
    image = widgets.Image(
        value=Image(filename=f"/{DATA_DIR}/{files[0]}").data,
        format="webp",
        width=300,
        height=300
    )
    
    text = widgets.Text(f"Total labeled: {len(labels)}")
    
    display(widgets.VBox([buttons, text, image, out]))
    
    
# show_widgets()

In [8]:
# labels.to_csv("new_labels.csv", index=False)

In [9]:
# This cell was ran separately for both test and train

def move_files(row):
    filename = row["filename"]
    label = row["label"].replace(" ", "_")
    
    os.rename(f"data/{filename}", f"{DATA_DIR}/train/{label}/{filename}")

faces = unmoved[unmoved["label"] == 'face']
no_faces = unmoved[unmoved["label"] == 'no face']

try:
    faces.apply(move_files, axis=1)
    no_faces.apply(move_files, axis=1)
    
    print("Moved files to relevant folders")
except:
    print("Images are already moved")

Moved files to relevant folders


In [16]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
])

tensor_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])


def collate_fn(batch):
    images, labels = zip(*batch)
    return list(images), list(labels)

dataset_test = datasets.ImageFolder(f"{DATA_DIR}/test", transform=transform)
dataset_train = datasets.ImageFolder(f"{DATA_DIR}/train", transform=transform)
loader_test = DataLoader(dataset_test, collate_fn=collate_fn, batch_size=8, shuffle=True)
loader_train = DataLoader(dataset_train, collate_fn=collate_fn, batch_size=8, shuffle=True,)

idx_to_class = {i:c for c, i in dataset_test.class_to_idx.items()}

tensor_dataset_test = datasets.ImageFolder(f'{DATA_DIR}/test', transform=tensor_transform)
tensor_dataset_train = datasets.ImageFolder(f'{DATA_DIR}/train', transform=tensor_transform)
tensor_loader_test = DataLoader(tensor_dataset_test, collate_fn=collate_fn, batch_size=8, shuffle=True)
tensor_loader_train = DataLoader(tensor_dataset_train, collate_fn=collate_fn, batch_size=8, shuffle=True)

# Define MTCNN baseline
We use the default params for now

In [12]:
# Define model with default hyperparameters
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [13]:
count = 0
test_count = 0

for X, Y in tqdm(loader_test):
    for i in range(0, len(X) - 1):
        x = X[i]
        y = Y[i]

        x_aligned, probs = mtcnn.detect(x)

        if x_aligned is not None and y == 0:
            count += 1

            if probs[0] > 0.75:
                test_count += 1
    
print(f"{(count / len(faces)):8f}")
print(f"{(test_count / len(faces)):8f}")

100%|██████████| 1251/1251 [04:54<00:00,  4.24it/s]

0.077838
0.077097


In [14]:
checkpoint = 1000
faulty_img = 0

for idx, file in enumerate(tqdm(files)):
    img = PIL.Image.open(f"{DATA_DIR}/{file}")
 
    try:
        # For some reason, there are images with 4 dimensions instead of 3.
        # We skip these images and continue with labeling
        faces, probs = mtcnn.detect(img)
        d = {'filename': file, 'label': 'face' if faces is not None else 'no face'}

        labels.loc[len(labels)] = d

        if ((idx + 1) % checkpoint) == 0:
            print(f"Saving at {idx + 1}")
            labels.to_csv("new_labels.csv", index=False)
    except RuntimeError:
        faulty_img += 1
        print(f"Found {faulty_img} faulty images so far")

 62%|██████▎   | 5/8 [00:00<00:00, 10.51it/s]

Found 1 faulty images so far
Found 2 faulty images so far
Found 3 faulty images so far
Found 4 faulty images so far
Found 5 faulty images so far
Found 6 faulty images so far
Found 7 faulty images so far
Found 8 faulty images so far


100%|██████████| 8/8 [00:00<00:00, 11.88it/s]


In [50]:
labels.to_csv("new_labels.csv", index=False)

In [19]:
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

In [17]:
# digits = load_digits()
X, y = [], []

for images, labels in tqdm(tensor_loader_test):
    # Flatten images to shape
    images_flat = [img.numpy().transpose(1, 2, 0).flatten() for img in images]
    X.extend(images_flat)
    y.extend(labels)

100%|██████████| 1251/1251 [02:14<00:00,  9.33it/s]


In [20]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = np.asarray(y, dtype=int)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

n_classes = len(np.unique(y_train))
gmm_models = []

In [21]:
for label in tqdm(range(n_classes)):
    X_class = X_train[y_train == label]
    
    gmm = GaussianMixture(n_components=n_classes, covariance_type='full', random_state=42)
    gmm.fit(X_class)
    gmm_models.append(gmm)

# gmm = GaussianMixture(n_components=len(idx_to_class), random_state=42)
# gmm.fit(X_train)

# # Step 5: Predict Labels
# y_pred = gmm.predict(X_test)

100%|██████████| 2/2 [09:11<00:00, 275.84s/it]


In [35]:
_, class_counts = np.unique(y_train, return_counts=True)

f"{class_counts[0] / len(y_train)} - {idx_to_class[0]} and {class_counts[1] / len(y_train)} - {idx_to_class[1]}"

'0.524 - face and 0.476 - no_face'

In [25]:
y_pred = []

for sample in tqdm(X_test):
    likelihoods = [gmm.score_samples(sample.reshape(1, -1)) for gmm in gmm_models]
    
    # Assign the class with the highest likelihood
    y_pred.append(np.argmax(likelihoods))
    
    
accuracy = accuracy_score(y_test, y_pred)

accuracy

100%|██████████| 3001/3001 [28:27<00:00,  1.76it/s]


0.5518160613128957